In [7]:
import pandas as pd
from datetime import datetime, timedelta
from random import randrange
from pandasql import sqldf
import awswrangler as wr

In [2]:
#df_new_file = pd.read_csv('s3://stock-market-raw-data-us-east-1/stg_price_by_date/test_data.csv')
df_new_file = pd.read_csv('s3://stock-market-raw-data-us-east-1/stg_price_by_date/20250131175440.csv')

In [5]:
df_athena = wr.athena.read_sql_query('SELECT * FROM price_by_date', database = 'stock_market')

In [533]:
df_new_records = sqldf(
         '''select a.company company, a.date as close_date, a.close_price   
            from df_new_file a 
            left join df_athena b on a.company = b.company and 
                a.date = b.close_date
            where b.company is null
         '''
     )

In [534]:
#convert string to date python date object

df_new_records['close_date'] = df_new_records['close_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 

#OR

#df_new_records['close_date'] = pd.to_datetime(df_new_records['close_date']).dt.date 

In [535]:
#create the partition column
df_new_records['p_year'] = pd.to_datetime(df_new_records['close_date']).dt.year

In [536]:
# Convert the DataFrame to a PyArrow Table
table = pa.Table.from_pandas(df_new_records)

In [537]:
# Write to Parquet with partitioning by year
s3_path = 's3://stock-market-raw-data-us-east-1/price_by_date/'
pq.write_to_dataset(table, root_path = s3_path, partition_cols=['p_year'])

In [542]:
def process_batch_files(path):
    
    import pandas as pd
    from datetime import datetime, timedelta
    from random import randrange
    from pyathena import connect
    from pandasql import sqldf
    import pyarrow as pa
    import pyarrow.parquet as pq
    
    df_new_file = pd.read_csv(path)
    
    conn = connect(s3_staging_dir='s3://de-youtube-project-useast-dev/',region_name='us-east-1')
    df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)
    
    df_new_records = sqldf(
         '''select a.company company, a.date as close_date, a.close_price   
            from df_new_file a 
            left join df_athena b on a.company = b.company and 
                a.date = b.close_date
            where b.company is null
         '''
     )
    
    df_new_records['close_date'] = df_new_records['close_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 
    df_new_records['p_year'] = pd.to_datetime(df_new_records['close_date']).dt.year
    
    table = pa.Table.from_pandas(df_new_records)
    s3_path = 's3://stock-market-raw-data-us-east-1/price_by_date/'
    pq.write_to_dataset(table, root_path = s3_path, partition_cols=['p_year'])
 

In [543]:
path = 's3://stock-market-raw-data-us-east-1/stg_price_by_date/20250131163557.csv'
process_batch_files(path)

<ipython-input-542-14245a7c672c>:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)


In [ ]:
s3://stock-market-raw-data-us-east-1/stg_price_by_date/20250131172624.csv

In [11]:
def process_batch_files_lambda(event, context):
    
    import pandas as pd
    from datetime import datetime, timedelta
    from random import randrange
    from pandasql import sqldf
    import awswrangler as wr

    
    source_bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']
    path = source_bucket + key
    
    df_new_file = pd.read_csv(path)
    
    df_athena = wr.athena.read_sql_query('SELECT * FROM price_by_date', database = 'stock_market')
    
    df_new_records = sqldf(
         '''select a.company company, a.date as close_date, a.close_price   
            from df_new_file a 
            left join df_athena b on a.company = b.company and 
                a.date = b.close_date
            where b.company is null
         '''
     )
    
    df_new_records['close_date'] = df_new_records['close_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 
    df_new_records['p_year'] = pd.to_datetime(df_new_records['close_date']).dt.year
    
    df_new_records.to_parquet(path = 's3://stock-market-raw-data-us-east-1/price_by_date/', index = False, \
                                partition_cols=['p_year'])
 

In [15]:
event = \
{  
   "Records":[  
      {  
         "eventVersion":"2.2",
         "eventSource":"aws:s3",
         "awsRegion":"us-west-2",
         "eventTime":"The time, in ISO-8601 format, for example, 1970-01-01T00:00:00.000Z, when Amazon S3 finished processing the request",
         "eventName":"event-type",
         "userIdentity":{  
            "principalId":"Amazon-customer-ID-of-the-user-who-caused-the-event"
         },
         "requestParameters":{  
            "sourceIPAddress":"ip-address-where-request-came-from"
         },
         "responseElements":{  
            "x-amz-request-id":"Amazon S3 generated request ID",
            "x-amz-id-2":"Amazon S3 host that processed the request"
         },
         "s3":{  
            "s3SchemaVersion":"1.0",
            "configurationId":"ID found in the bucket notification configuration",
            "bucket":{  
               "name":"s3://stock-market-raw-data-us-east-1/",
               "ownerIdentity":{  
                  "principalId":"Amazon-customer-ID-of-the-bucket-owner"
               },
               "arn":"arn:aws:s3:::stock-market-raw-data-us-east-1"
            },
            "object":{  
               "key":"stg_price_by_date/20250131192045.csv",
               "size":"object-size in bytes",
               "eTag":"object eTag",
               "versionId":"object version if bucket is versioning-enabled, otherwise null",
               "sequencer": "a string representation of a hexadecimal value used to determine event sequence, only used with PUTs and DELETEs"
            }
         },
         "glacierEventData": {
            "restoreEventData": {
               "lifecycleRestorationExpiryTime": "The time, in ISO-8601 format, for example, 1970-01-01T00:00:00.000Z, of Restore Expiry",
               "lifecycleRestoreStorageClass": "Source storage class for restore"
            }
         }
      }
   ]
}

In [14]:
context = ''
process_batch_files_lambda(event, context)